In [3]:
from services import VectorSimilarityService
from augmentation import FederatedGraphVectorizer, GraphQLSource
from mars_vectorizer_sdk import Vectorizer, Property
from starlette.config import Config
from authlib.integrations.requests_client import OAuth2Session
from dataclasses import dataclass
from typing import List
import numpy as np
config = Config(".env")
from mars_vectorizer_sdk import GroupParser, PropertyParser, ListPropertyParser
from dataclasses import dataclass
from hashlib import sha256
from typing import List, Optional, Callable
from storages import LocalStorage
from functools import reduce
import grpc
import vectorization_pb2
import vectorization_pb2_grpc
@dataclass
class GQLQuery:

    query: str          # The GQL query (not the body of the query, just the name of it).
    query_input: str    # The argument input name for the query.
    items_type: str     # The GQL input type for the items.
    fields: str         # Body of the query
    data_path: list     # Path to the data in the response
    inject: Callable[[dict], dict] = lambda x: x

    def __call__(self, items: List[dict]) -> dict:
        return {
            "query": f"""
                query Query($items: [{self.items_type}]!) {{
                    {self.query}({self.query_input}: $items) {{
                        {self.fields}  
                    }}
                }}
            """,
            "variables": {
                "items": list(map(self.inject, items)),
            }
        }
    
    def sha256(self) -> str:
        return sha256(f"{self.query}{self.query_input}{self.items_type}{self.fields}".encode()).hexdigest()
    
    def parse_response(self, response: dict) -> list:
        return reduce(lambda x, y: x[y], self.data_path, response)
@dataclass
class Bias:

    name: str
    value: float

    def sha256(self) -> str:
        return sha256(f"{self.name}{self.value}".encode()).hexdigest()

@dataclass
class BiasProfile:

    biases: List[Bias]

    def sha256(self) -> str:
        return sha256("".join(map(lambda x: x.sha256(), sorted(self.biases, key=lambda x: x.name))).encode()).hexdigest()

@dataclass
class VectorizationProfile:

    query:  GQLQuery
    group:  GroupParser
    bias:   BiasProfile

    def sha256(self) -> str:
        return sha256(f"{self.query.sha256()}{self.group.sha256()}{self.bias.sha256()}".encode()).hexdigest()

class ProfileManager:

    @classmethod
    def bias(cls, profile_id: str) -> BiasProfile:
        return {
            "default": BiasProfile([]),
            "basics": BiasProfile([
                Bias("car type code", 10),
            ]),
            "design": BiasProfile([
                Bias("color name", 10),
                Bias("color description", 5),
                Bias("upholstery name", 2),
                Bias("upholstery description", 2),
            ]),
            "performance": BiasProfile([
                Bias("engine name", 10),
                Bias("engine description", 5),
            ]),
            "features": BiasProfile([
                Bias("option name", 10),
            ]),
        }.get(profile_id, BiasProfile([]))
    
    @classmethod
    def profiles(cls, profile_id: str) -> Optional[VectorizationProfile]:
        return next(
            map(
                lambda x: x[1],
                filter(
                    lambda x: profile_id in x[0],
                    [
                        (("default", "basics", "design", "performance", "features"), VectorizationProfile(
                            query=GQLQuery(
                                query="carsByCarKeys",
                                query_input="carKeys",
                                items_type="CarKeyInput!",
                                fields="""
                                    carType {
                                        code
                                        content(locale: "en-GB") {
                                            displayName {
                                                value
                                            }
                                        }
                                    }
                                    color {
                                        code
                                        content(locale: "en-GB") {
                                            displayName {
                                                value
                                            }
                                            shortDescription {
                                                value
                                            }
                                        }
                                    }
                                    engine {
                                        code
                                        content(locale: "en-GB") {
                                            displayName {
                                                value
                                            }
                                            shortDescription {
                                                value
                                            }
                                        }
                                    }
                                    salesVersion {
                                        code
                                        globalName
                                    }
                                    upholstery {
                                        code
                                        content(locale: "en-GB") {
                                            displayName {
                                                value
                                            }
                                            shortDescription {
                                                value
                                            }
                                        }
                                    }
                                    options {
                                        content(locale: "en-GB") {
                                            displayName {
                                                value
                                            }
                                        }
                                    }
                                """,
                                data_path=["data", "carsByCarKeys"],
                                inject=lambda x: x["car_key"]
                            ),
                            group=GroupParser(
                                name="car",
                                children=[
                                    GroupParser(
                                        name="model",
                                        children=[
                                            PropertyParser("car type code", str, ["carType", "code"]),
                                        ]
                                    ),
                                    GroupParser(
                                        name="color",
                                        children=[
                                            PropertyParser("color name", str, ["color", "content", "displayName", "value"]),
                                            PropertyParser("color description", str, ["color", "content", "shortDescription", "value"]),
                                        ]
                                    ),
                                    GroupParser(
                                        name="engine",
                                        children=[
                                            PropertyParser("engine name", str, ["engine", "content", "displayName", "value"]),
                                            PropertyParser("engine description", str, ["engine", "content", "content", "shortDescription", "value"]),
                                        ]
                                    ),
                                    GroupParser(
                                        name="sales version",
                                        children=[
                                            PropertyParser("sales version code", str, ["salesVersion", "code"]),
                                            PropertyParser("sales version name", str, ["salesVersion", "globalName"]),
                                        ]
                                    ),
                                    GroupParser(
                                        name="upholstery",
                                        children=[
                                            PropertyParser("upholstery name", str, ["upholstery", "content", "displayName", "value"]),
                                            PropertyParser("upholstery description", str, ["upholstery", "content", "shortDescription", "value"]),
                                        ]
                                    ),
                                    ListPropertyParser(
                                        name="options",
                                        parser=PropertyParser("option name", str, ["content", "displayName", "value"]),
                                        path=["options"],
                                    ),
                                ]
                            ),
                            bias=BiasProfile(
                                biases=[]
                            )
                        ))
                    ]
                ), 
            ),
            None
        )

    @classmethod
    def get(cls, profile_id: str) -> Optional[VectorizationProfile]:
        profile = cls.profiles(profile_id)
        if profile is not None:
            profile.bias = cls.bias(profile_id)
            return profile
        return None
@dataclass
class GrpcVectorizer(Vectorizer):

    host: str
    port: int

    key_coef: float = 1_000_000

    def vectorize(self, key_values: List[Property]) -> np.ndarray:
        with grpc.insecure_channel(f"{self.host}:{self.port}") as channel:
            stub = vectorization_pb2_grpc.VectorizationServiceStub(channel)
            return np.array(
                list(
                    map(
                        lambda x: x.value, 
                        stub.Vectorize(
                            vectorization_pb2.VectorizeRequestSet(
                                key_value_pairs=vectorization_pb2.KeyValuePairSet(
                                    key_values=list(
                                        map(
                                            lambda property: vectorization_pb2.KeyValuePair(
                                                key=property.name,
                                                value=vectorization_pb2.StringOrFloatValue(
                                                    string_value=property.value,
                                                ) if isinstance(property.value, str) else vectorization_pb2.StringOrFloatValue(
                                                    float_value=float(property.value) if isinstance(property.value, float) else 0.0
                                                )
                                            ),
                                            key_values
                                        )
                                    )
                                ),
                                setting=vectorization_pb2.Setting(
                                    key_coefficient=self.key_coef,
                                    string_val_coefficient=1,
                                    float_val_coefficient=1,
                                )
                            )
                        ).vectors
                    )
                )
            )
profile = ProfileManager.get("default")
vssp = VectorSimilarityService(
    augmentor=FederatedGraphVectorizer(
                vectorizer=GrpcVectorizer(
                    host="localhost",
                    port=8000,
                ),
                query=profile.query,
                parser=profile.group,
                source=GraphQLSource(
                    url=config("FED_GRAPH_URL"),
                    headers={
        "apollographql-client-name": config("FED_GRAPH_CALLER"),
    },
                ),
                batch_size=10,
                timeout=1.0,
                cache=LocalStorage(),
            )
)

In [4]:
@dataclass
class CarKeyInput:
    body: str
    engine: str
    gearbox: str
    marketingCode: str
    modelYear: float
    salesVersion: str
    specMarket: str
    steering: str
    structureWeek: float
    carType: Optional[str]          = None
    color: Optional[str]            = None
    upholstery: Optional[str]       = None
    options: Optional[List[str]]    = None
    packages: Optional[List[str]]   = None
    sMessage: Optional[str]         = None

    def sha256(self) -> str:
        return hashlib.sha256(
            json.dumps(self.__dict__).encode()
        ).hexdigest()
    
    def to_dict(self):
        return {"car_key": self.__dict__}
    
    def from_dict(self, dict):
        return "Success"



In [6]:

vssp.similarity_search([ CarKeyInput(carType="256", body="0", engine="29", gearbox="0", marketingCode="10", modelYear=2016, salesVersion="13", specMarket="110", steering="1", structureWeek=201601)], CarKeyInput(carType="256", body="0", engine="29", gearbox="0", marketingCode="10", modelYear=2016, salesVersion="13", specMarket="110", steering="1", structureWeek=201601))

/opt/anaconda3/envs/customer_cars/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/customer_cars/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[SimilarityResult(score=0.0, obj='Success')]